# DenseNet Application of Image Classification

In [1]:
from torchvision.models.densenet import densenet121 as feature_extractor
from torch import nn
import torch

## Config

In [2]:
import os

In [3]:
SCALE = 256
TRAIN = "/data/avatar/train/"
VALID = "/data/avatar/test/"
CUDA = torch.cuda.is_available()
DENSE_FEATURE = 1024
BS = 32
VERSION = "0.0.1"

In [4]:
conv_model = feature_extractor(pretrained=True)

/home/zhangxiaochen/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


### Split ConvLayer to 2 parts, conv0~transtition3, (not gonna train), denseblock4~norm5 (train)

In [5]:
dense_conv1 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["conv0","norm0","relu0","pool0","denseblock1","transition1",
                                                                                   "denseblock2","transition2","denseblock3","transition3",]])

dense_conv2 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["denseblock4","norm5"]])

### Loading data

In [6]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import functional as F

In [7]:
transform = transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomAffine([-10,10]), 
                                transforms.Resize((SCALE,SCALE)),
                                transforms.ToTensor(),
                                transforms.Normalize([.5,.5,.5],[.5,.5,.5]),
                               ])

trn = ImageFolder(TRAIN,transform=transform)
val = ImageFolder(VALID,transform=transform)

In [9]:
CLASS_TO_IDX = trn.class_to_idx
IDX_TO_CLASS = dict((v,k) for k,v in CLASS_TO_IDX.items())
print(IDX_TO_CLASS)

{0: 'erotic', 1: 'normal', 2: 'poli', 3: 'porn', 4: 'qr', 5: 'w_normal', 6: 'w_risk'}


In [10]:
# data_gen = iter(DataLoader(trn,shuffle=True))

# next(data_gen)

### Top half of the model, with fully connected layers

In [11]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten,self).__init__()
    def forward(self,x):
        bs = x.size()[0]
        return x.view(bs,-1)

In [12]:
fl = Flatten()
avg_pool = nn.AdaptiveAvgPool2d((1,1))
FEATURE_WIDTH = fl(avg_pool(dense_conv2(dense_conv1(torch.rand(2,3,SCALE,SCALE))))).size()[1]
print(FEATURE_WIDTH)

1024


In [13]:
class top_half(nn.Module):
    def __init__(self):
        super(top_half,self).__init__()
        self.top_ = nn.Sequential(*[nn.AdaptiveAvgPool2d((1,1)),Flatten(),
                                   nn.Linear(FEATURE_WIDTH,DENSE_FEATURE,bias=False),
                                    nn.BatchNorm1d(DENSE_FEATURE),
                                    nn.LeakyReLU(inplace=True),
                                    nn.Dropout(p=.5),
                                    nn.Linear(DENSE_FEATURE,len(CLASS_TO_IDX),bias=True),
                                   ])
    def forward(self,x):
        return F.softmax(self.top_(x))

### Construct Model, optimizer,train function

In [14]:
top_half_ = top_half()

In [15]:
if CUDA:
    top_half_.cuda()
    dense_conv1.cuda()
    dense_conv2.cuda()
    
from torch.optim import Adam

opt = Adam(list(dense_conv2.parameters())+list(top_half_.parameters()),amsgrad=True)
loss_func = nn.CrossEntropyLoss()

In [16]:
from p3self.matchbox import Trainer

In [17]:
trainer = Trainer(trn,val_dataset=val,batch_size=BS,print_on=5,)

In [18]:
def argmax(x):
    return torch.max(x,dim=1)[1]

def accuracy(y_pred,y_true):
    return (argmax(y_pred)==y_true).float().mean()

def save_model(model,path):
    """
    model:pytorch model
    path:save to path, end with pkl
    """
    torch.save(model.state_dict(), path)
    
def load_model(model,path):
    model.load_state_dict(torch.load(path))

def save_():
    save_model(dense_conv2,"/data/weights/dense_conv2.%s.pkl"%(VERSION))
    save_model(top_half_,"/data/weights/top_half.%s.pkl"%(VERSION))
    
def load_():
    load_model(dense_conv2,"/data/weights/dense_conv2.%s.pkl"%(VERSION))
    load_model(top_half_,"/data/weights/top_half.%s.pkl"%(VERSION))
    

In [19]:
def action(*args,**kwargs):
    x,y = args[0]
    if CUDA:
        x,y = x.cuda(),y.cuda()
    x = x[:,:3,...]
    opt.zero_grad()
    y_ = top_half_(dense_conv2(dense_conv1(x)))
    
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)
    
    loss.backward()
    opt.step()
    if kwargs["ite"]%10==9:
        save_()
    return {
        "loss":loss.item(),
        "acc":acc.item(),
    }
    
def val_action(*args,**kwargs):
    x,y = args[0]
    if CUDA:
        x,y = x.cuda(),y.cuda()
    x = x[:,:3,...]
    y_ = top_half_(dense_conv2(dense_conv1(x)))
    
    loss = loss_func(y_,y)
    acc = accuracy(y_,y)

    return {
        "loss":loss.item(),
        "acc":acc.item(),
    }

In [ ]:
trainer.action = action
trainer.val_action = val_action

In [ ]:
trainer.train(20)

  0%|          | 0/2408 [00:00<?, ?it/s]/home/zhangxiaochen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
⭐[ep_0_i_129]	acc	0.919✨	loss	1.335:   6%|▌         | 133/2408 [01:14<21:16,  1.78it/s]/home/zhangxiaochen/anaconda3/lib/python3.6/site-packages/PIL/Image.py:885: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
⭐[ep_0_i_1839]	acc	1.000✨	loss	1.165:  76%|███████▋  | 1841/2408 [16:02<04:56,  1.91it/s]